In [1]:
import numpy as np
from itertools import combinations, product


In [46]:
def hs(mat1, mat2):
    return np.trace(mat1.getH()*mat2)

def norm(mat1):
    return mat1/np.sqrt(np.abs(hs(mat1, mat1)))

In [6]:
third = smp.N(second - hs(first, second)*first)

In [176]:
dim = 3 # Dimension

def modadd(state, Z_or_X, index):
    if state[Z_or_X][index] == dim-1:
        state[Z_or_X][index]=0
    else:
        state[Z_or_X][index]+=1
    return state

def evolution(state, size):
    Z_part, X_part = state
    new_state = [[0 for _ in range(size)],[0 for _ in range(size)]]
    # X to Z evolution
    for i in range(size):
        for k in range(X_part[i]):
            # Z
            new_state = modadd(new_state, 0, i)
            new_state = modadd(new_state, 0, i)
    # Z evolution
    for i in range(size):
        for k in range(Z_part[i]):
            if i == 0: # Boundary
                # Z
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i+1)
                new_state = modadd(new_state, 0, i+1)
                # X
                new_state = modadd(new_state, 1, i)
            elif i < size-1:
                # Z
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i+1)
                new_state = modadd(new_state, 0, i+1)
                # X
                new_state = modadd(new_state, 1, i)
            else:
                # Z
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                # X
                new_state = modadd(new_state, 1, i) 
    return new_state

state_0=[[1, 0, 0],[0, 0, 0]] # (Z, X)

size = len(state_0[0])

state = evolution(state_0, size)

all_states = [state_0]

while state != state_0:
    all_states.append(state)
    state = evolution(state, size)
print(len(all_states))


30


In [177]:
I = np.matrix([[1,0,0], [0,1,0], [0,0,1]])
X = np.matrix([[0,0,1],[1,0,0], [0,1,0]])
Z = np.matrix([[1,0,0], [0,np.e**(1j*2*np.pi/3),0], [0, 0, np.e**(1j*4*np.pi/3)]])


H = (1/np.sqrt(3))*np.matrix([[1,1,1],[1,np.e**(1j*2*np.pi/3),np.e**(1j*4*np.pi/3)], [1, np.e**(1j*4*np.pi/3), np.e**(1j*2*np.pi/3)]])
S = np.matrix([[np.e**(1j*2*np.pi/3),0,0], [0,1,0], [0, 0, 1]])
CZ = np.kron(np.matrix([[1,0, 0], [0,0, 0], [0, 0, 0]]), I)+np.kron(np.matrix([[0,0, 0], [0,1, 0], [0, 0, 0]]), Z) + np.kron(np.matrix([[0,0, 0], [0,0, 0], [0, 0, 1]]), Z**2)



T_3 = np.kron(H*S, np.kron(H*S, H*S))*np.kron(CZ, I)*np.kron(I, CZ)
T_2 = np.kron(H*S, H*S)*CZ



In [178]:
def round2zero(m, e):
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            if np.abs(np.imag(m[i,j])) < e:
                m[i,j] = m[i,j]-np.imag(m[i,j])*1j
            if np.abs(np.real(m[i,j])) < e:
                m[i,j] = m[i,j]-np.real(m[i,j])
e = .0000001
#round2zero(A, e)

In [179]:
l3 = np.matrix([[1, 0, 0], [0, -1, 0], [0, 0, 0]])

l8 = np.matrix([[1, 0, 0], [0, 1, 0], [0, 0, -2]])*1/np.sqrt(3)


In [189]:
start = np.kron(l3, np.kron(I, I))
#start = np.kron(l8, np.kron(I, I))
ortho = []
ortho.append(start)

In [190]:
for i in range(1, len(all_states)):
    A = T_3.getH()**i*start*T_3**i
    A = norm(A)
    round2zero(A, e)
    ortho.append(A)

In [191]:
loop = True

new_length = len(ortho)

In [192]:
while loop == True:
    commutations = []
    comb1 = list(combinations([i for i in range(new_length)], 2))
    list1 = [i for i in range(new_length)]
    list2 = [i for i in range(new_length, len(ortho))]
    comb2 = list(product(list1,list2))
    total_comb = comb1 + comb2
    print("Length: ", len(total_comb), " list1: ", len(list1)\
         , " list2: ", len(list2))
    for i1, i2 in total_comb:
        conj = ortho[i1]*ortho[i2]-ortho[i2]*ortho[i1]
        round2zero(conj, e)
        if conj.any():
            commutations.append(norm(conj))
    total_count = 0
    for mat_c in commutations:
        count  = 0
        for i in range(len(ortho)):
            if np.abs(hs(ortho[i], mat_c)) > 0.85:
                count = 1
                break
            mat_c += -np.abs(hs(ortho[i], mat_c))*ortho[i]
            mat_c = norm(mat_c)
            round2zero(mat_c, e)
            if mat_c.any() == False:
                count = 1
                break
        mat_c = norm(mat_c)
        round2zero(mat_c, e)
        if count == 0:
            ortho.insert(0, mat_c)
            total_count+=1
            print(len(ortho))
    print(len(ortho))
    if total_count == 0:
        loop = False
    print('ortho: ', len(ortho), 'new: ', total_count)
    new_length = 20 # change to make faster, will be extremely large if len(new_ortho)

In [200]:
total_count = 0
for i in range(20, len(ortho)):
    for j in range(i+1, len(ortho)):
        if np.abs(hs(ortho[i], ortho[j]))>0.3:
            total_count += 1
            break

In [202]:
total_count

537

In [146]:
hs(ortho[-83], ortho[-82])

(0.9999999966281986-7.415332370048636e-06j)

In [66]:
check = ortho[0].copy()
round2zero(check, e)
check = norm(check)

"""
for i in range(1, len(ortho)):
    norm_check = norm(ortho[i])
    check += -np.abs((hs(norm_check, check)))*norm_check
"""

'\nfor i in range(1, len(ortho)):\n    norm_check = norm(ortho[i])\n    check += -np.abs((hs(norm_check, check)))*norm_check\n'

In [68]:
hs(check, norm_check)

(0.9996806433952741-6.227405873720734e-06j)

In [ ]:
norm_check = norm(ortho[1])
check-np.abs((hs(norm_check, check)))*norm_check

In [ ]:
hs(ortho[i], mat_c)*ortho[i]

In [23]:
check = ortho[0]
for i1 in range(len(ortho)):
    if np.abs(hs(ortho[i1],check))>e:
        print(i1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
71
74
77


In [ ]:
for i1 in range(len(ortho)):
    for i2 in range(len(ortho)):
        if i1 != i2:
            if np.abs(hs(ortho[i1],ortho[i2]))>e:
                print(i1, i2)

In [351]:
comm = []
for i1 in range(len(ortho)):
    for i2 in range(len(ortho)):
        if i1 != i2:
            conj = ortho[i1]*ortho[i2]-ortho[i2]*ortho[i1]
            round2zero(conj, e)
            if conj.any():
                comm.append(conj)

In [365]:
gram_comm = []
for j in comm:
    mat_c = j.copy()
    for i in range(len(ortho)):
        mat_c += -hs(mat_c, ortho[i])*ortho[i]
        if mat_c.any() == False:
            break
    for i in range(len(gram_comm)):
        mat_c += -hs(mat_c, gram_comm[i])*gram_comm[i]
        if mat_c.any() == False:
            break
    mat_c = norm(mat_c)
    round2zero(mat_c, e)
    if mat_c.any():
        gram_comm.append(mat_c)
        

In [ ]:
for i1 in range(len(gram_comm)):
    for i2 in range(len(gram_comm)):
        if i1 != i2:
            if np.abs(hs(gram_comm[i1], gram_comm[i2])) >e:
                print(i1, i2)